In [0]:
retailDataDay = "dbfs:/mnt/data/data/retail-data/by-day/"
retailDataDaySmall = "dbfs:/mnt/data/data/retail-data/by-day/2011-12-06.csv"

### Note remove all the data from data/retail-data/by-day/ and keep only 1 csv file - 2011-12-06.csv
 - Need to remove manually

In [0]:
staticDataFrame = spark\
.read\
.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(retailDataDaySmall)

In [0]:
staticSchema = staticDataFrame.schema

In [0]:
# maxFilesPerTrigger option simply specifies the number of files we should read in at once
# This is to make our demonstration more “streaming,” and in a production scenario this would probably be omitted

streamingDataFrame = spark\
.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load(retailDataDay)

In [0]:
from pyspark.sql.functions import col, window

In [0]:
purchaseByCustomer = streamingDataFrame\
.selectExpr(\
           "CustomerId",
           "(UnitPrice * Quantity) as total_cost",\
           "InvoiceDate")\
.groupBy(\
        col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

In [0]:
purchaseByCustomer.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()

In [0]:
spark.sql("""
select *
from customer_purchases
order by 'sum(total_cost)' desc""")\
.count()

0

In [0]:
purchaseByCustomer.writeStream.trigger(processingTime='10 seconds')\
  .format("console").outputMode("complete").start()

### Once the files are processed, copy 1 CSV file each time manually to data/retail-data/by-day/
- Notice that processing is again started